In [1]:
import keras
from keras.layers import Dense , Flatten , MaxPooling2D , Conv2D , Input
from keras.models import Sequential,load_model , Model
from keras.losses import SparseCategoricalCrossentropy
from keras.activations import relu
from keras.optimizers import Adam
from keras.applications import VGG16
from keras.preprocessing.image import load_img , array_to_img , ImageDataGenerator
import matplotlib.pyplot as plt

In [2]:
!pip install kaggle

In [3]:
from google.colab import files
upload = files.upload()

Saving kaggle.json to kaggle.json


In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [5]:
!kaggle datasets download -d sujaykapadnis/monkeypox-skin-images-dataset-msid
!unzip -q monkeypox-skin-images-dataset-msid

 95% 52.0M/54.6M [00:00<00:00, 96.2MB/s]
100% 54.6M/54.6M [00:00<00:00, 85.7MB/s]


In [6]:
dataPath = '/content/Monkeypox Skin Images Dataset (MSID)/Monkeypox Skin Image Dataset/Monkeypox Skin Image Dataset'
img_height = 224
img_width = 224
dataGen = ImageDataGenerator(
    rescale=1/255.,
    validation_split = 0.2
)
TrainData = dataGen.flow_from_directory(
    dataPath,
    target_size=((img_height, img_width)),
    class_mode='categorical',
    batch_size=32,
    subset='training',
)
validationData = dataGen.flow_from_directory(
    dataPath,
    target_size=((img_height, img_width)),
    class_mode='categorical',
    batch_size=32,
    subset='validation',
)

Found 618 images belonging to 4 classes.
Found 152 images belonging to 4 classes.


In [7]:
baseModel = VGG16(include_top=False, weights='imagenet', input_shape=((img_height, img_width, 3)))

for layer in baseModel.layers:
  layer.trainable = False

input_layer = Input(shape=(224,224,3))
modelOuput = baseModel(input_layer) # Pass the input through the VGG16 base model
fOutput = Flatten()(modelOuput) # Flatten the output and add additional fully connected layers
dense_1 = Dense(512, activation='relu')(fOutput)
dense2 = Dense(256, activation='relu')(dense_1)
output_layer = Dense(4, activation='softmax')(dense2)


58889256/58889256 [==============================] - 0s 0us/step


In [8]:
model = Model(inputs=input_layer, outputs=output_layer)
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 512)               12845568  
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 4)                 1028      
                                                                 
Total params: 27692612 (105.64 MB)
Trainable params: 12977924

In [10]:
history = model.fit(
        TrainData ,
        epochs = 50,
        batch_size = 128,
        validation_data =validationData
)

Epoch 1/50
20/20 [==============================] - 24s 628ms/step - loss: 3.0373 - accuracy: 0.4304 - val_loss: 1.4005 - val_accuracy: 0.5132
Epoch 2/50
20/20 [==============================] - 3s 168ms/step - loss: 0.8077 - accuracy: 0.7120 - val_loss: 1.0833 - val_accuracy: 0.5987
Epoch 3/50
20/20 [==============================] - 4s 176ms/step - loss: 0.4508 - accuracy: 0.8430 - val_loss: 0.8358 - val_accuracy: 0.6711
Epoch 4/50
20/20 [==============================] - 3s 177ms/step - loss: 0.2890 - accuracy: 0.8900 - val_loss: 0.9020 - val_accuracy: 0.6908
Epoch 5/50
20/20 [==============================] - 3s 172ms/step - loss: 0.1898 - accuracy: 0.9466 - val_loss: 0.7608 - val_accuracy: 0.7500
Epoch 6/50
20/20 [==============================] - 4s 203ms/step - loss: 0.1168 - accuracy: 0.9693 - val_loss: 0.6543 - val_accuracy: 0.8026
Epoch 7/50
20/20 [==============================] - 4s 176ms/step - loss: 0.0862 - accuracy: 0.9806 - val_loss: 0.7013 - val_accuracy: 0.7961
Epoch

In [ ]:
model.save("VGG16.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
